<a href="https://colab.research.google.com/github/asafdari-boop/ComputationalPhyiscsLibrary/blob/main/ODE3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The shooting method

Throw a ball up with an initial velocity $v_0$:

$\frac{dx}{dt} = v $

$\frac{dv}{dt} = -g$

convert a boundary value problem to an initial value problem and then find the root of a nonlinear equation.

Example:

$$
\frac{d^2x}{dt^2} = f(\frac{dx}{dt}, x, t)
$$
with $a \le t \le b$, $x(t=a)=\alpha$ and $x(t=b)=\beta$. 

We can convert this problem to
$$
\frac{d^2x}{dt^2} = f(\frac{dx}{dt}, x, t)
$$
with  $x(t=a)=\alpha$ and $\frac{dx}{dt}(t=a)=v$. 

The goal is to determine an appropriate value $v$ for the initial slope so that the solution of the initial value problem is also a solution of the boundary value problem. 

If we define a function $h(v)=x(t=b, v)-\beta$, we need to find the root. If we use the Newton's mthod:

$$
v_{n+1} = v_n - \frac{h(v_n)}{h'_{v_n}} = v_n - \frac{x(t=b, v_n)-\beta}{\frac{d}{dt}x(t=b, v)|_{v_{n}}} = v_n-\frac{[x(t=b, v_n)-\beta](v_n-v_{n-1})}{x(t=b, v_n)-x(t=b, v_{n-1})}
$$


In [ ]:
from numpy import array,arange

g = 9.81         # Acceleration due to gravity
a = 0.0          # Initial time
b = 10.0         # Final time
N = 1000         # Number of Runge-Kutta steps
h = (b-a)/N      # Size of Runge-Kutta steps
target = 1e-10   # Target accuracy for binary search

# Function for Runge-Kutta calculation
def f(r):
    x = r[0]
    v = r[1]
    fx = v
    fy = -g
    return array([fx, fy], float)

# Function to calculate the final height given an initial speed v0
def height(v0):
    # set initial height to 0 and the initial velocity to v0
    r = array([0.0, v0], float)

    for t in arange(a,b,h):
        k1 = h*f(r)
        k2 = h*f(r+0.5*k1)
        k3 = h*f(r+0.5*k2)
        k4 = h*f(r+k3)
        r += (k1+2*k2+2*k3+k4)/6
    return r[0]   # final height 

# Main program performs a binary search
# the velocity we need will be between the range [v1, v2]
# need to find a velocity so that the predicted height agrees with the height 0 we want to pass
v1 = 0.01     # search range
v2 = 1000.0   # search range
h1 = height(v1)
h2 = height(v2)

while abs(h2-h1)>target:
    vp = (v1+v2)/2
    hp = height(vp)
    if h1*hp>0:
        v1 = vp
        h1 = hp
    else:
        v2 = vp
        h2 = hp

v = (v1+v2)/2
print("The required initial velocity is",v,"m/s")

The required initial velocity is 49.04999999999815 m/s


# Eigenvalue for a square well
$$
-\frac{\hbar^2}{2m} \frac{d^2 \psi(x)}{dt^2} + V \psi(x) = E \psi(x)
$$
with $V=0$ for $0<x<L$ and $\infty$ elsewhere, $\psi(x=0)=\psi(x=L)=0$

Define $\frac{d\psi}{dt}=\phi$ and $\frac{d\phi}{dt} = \frac{2m}{\hbar^2} [V-E]\psi$

In [ ]:
from numpy import array,arange

# Constants
m = 9.1094e-31     # Mass of electron
hbar = 1.0546e-34  # Planck's constant over 2*pi
e = 1.6022e-19     # Electron charge
L = 5.2918e-11     # Bohr radius
N = 1000
h = L/N

# Potential function
def V(x):
    return 0.0

# the above two differential equations
def f(r, x, E):
    psi = r[0]
    phi = r[1]
    fpsi = phi
    fphi = (2*m/hbar**2)*(V(x)-E)*psi
    return array([fpsi, fphi],float)

# Calculate the wavefunction for a particular energy
def solve(E):
    psi = 0.0
    phi = 1.0
    r = array([psi,phi], float)

    for x in arange(0, L, h):
        k1 = h*f(r, x, E)
        k2 = h*f(r+0.5*k1, x+0.5*h, E)
        k3 = h*f(r+0.5*k2, x+0.5*h, E)
        k4 = h*f(r+k3, x+h, E)
        r += (k1+2*k2+2*k3+k4)/6

    return r[0]

# Main program to find the energy using the secant method
# which is the root of the function solve(E)
E1 = 0.0
E2 = e
psi2 = solve(E1)

target = e/1000
while abs(E1-E2) > target:
    psi1, psi2 = psi2, solve(E2)
    E1, E2 = E2, E2-psi2*(E2-E1)/(psi2-psi1)

print("E =", E2/e, "eV")

E = 134.28637169369105 eV
